In [9]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [10]:
movies_df = pd.read_csv("../datasets/movies_df.csv", low_memory = False)

In [11]:
movies_df

,id,title,summary,rating,bucketed_rating,genres_str
0,42184,Eep!,Viegeltje looks like a girl but has wings inst...,5.900,1,Adventure|Family
1,539115,Náufragos,Odete tries to guess where her husband would b...,5.000,1,Drama
2,596063,CTRL+ALT+DLT,A hacker gets owned by a mysterious deep web e...,2.400,0,NonGiven
3,464253,Animelo Summer Live 2015 -THE GATE- 8.29,Second Day of Animelo Summer Live 2015 - THE G...,10.000,2,Music
4,188360,Who Flung Poo,An artist named Poo paints chicken pot pies wi...,5.500,1,Comedy
...,...,...,...,...,...,...
309113,16784,Welcome to Mooseport,A US president who has retired after two terms...,5.099,1,Comedy
309114,65199,Tokyo Train Girls: Private Lessons,New teacher Chihiro has been on the job for th...,5.000,1,Drama|Romance
309115,1043565,Mindcage,Detectives Jake Doyle and Mary Kelly seek the ...,6.127,1,Mystery|Thriller|Crime
309116,549791,The Lady's Driver,A Lady of the royal family live and breed on t...,5.300,1,Comedy|Drama
